In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import json
from pandas import json_normalize

# df_train = pd.read_json('/content/drive/MyDrive/NTPUIM/NTCIR/FinNUM 3/Dataset/FinNum-3_ConCall_train.json') #Results contain the required data
df_train = pd.read_json('/content/drive/MyDrive/FinNUM_dataset/FinNum-3_ConCall_train.json')
df_train

,paragraph,target_num,category,offset_start,offset_end,claim
0,Good day and welcome to the Apple Inc. Third Q...,2018.0,date,65,69,0
1,Please note that some of the information you'l...,10.0,other,504,506,0
2,Please note that some of the information you'l...,8.0,other,536,537,0
3,Thank you Nancy and thanks to everyone for joi...,53.3,money,212,216,0
4,Thank you Nancy and thanks to everyone for joi...,3.0,date,243,244,0
...,...,...,...,...,...,...
8332,Our momentum in pharmacy care services was exc...,2018.0,date,56,60,0
8333,Our momentum in pharmacy care services was exc...,98.0,quantity_relative,102,104,0
8334,Our momentum in pharmacy care services was exc...,2020.0,date,282,286,0
8335,In care delivery our clinical leaders are appl...,99.0,quantity_relative,212,214,0


In [ ]:
# df_test = pd.read_json('/content/drive/MyDrive/NTPUIM/NTCIR/FinNUM 3/Dataset/FinNum-3_ConCall_train.json') #Results contain the required data
df_test = pd.read_json('/content/drive/MyDrive/FinNUM_dataset/FinNum-3_ConCall_train.json')
df_test

,paragraph,target_num,category,offset_start,offset_end,claim
0,Good day and welcome to the Apple Inc. Third Q...,2018.0,date,65,69,0
1,Please note that some of the information you'l...,10.0,other,504,506,0
2,Please note that some of the information you'l...,8.0,other,536,537,0
3,Thank you Nancy and thanks to everyone for joi...,53.3,money,212,216,0
4,Thank you Nancy and thanks to everyone for joi...,3.0,date,243,244,0
...,...,...,...,...,...,...
8332,Our momentum in pharmacy care services was exc...,2018.0,date,56,60,0
8333,Our momentum in pharmacy care services was exc...,98.0,quantity_relative,102,104,0
8334,Our momentum in pharmacy care services was exc...,2020.0,date,282,286,0
8335,In care delivery our clinical leaders are appl...,99.0,quantity_relative,212,214,0


In [ ]:
pip install transformers

     |████████████████████████████████| 3.4 MB 5.4 MB/s 
     |████████████████████████████████| 596 kB 47.3 MB/s 
     |████████████████████████████████| 3.3 MB 48.9 MB/s 
     |████████████████████████████████| 895 kB 44.2 MB/s 
     |████████████████████████████████| 61 kB 492 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import RobertaModel, RobertaTokenizer
from transformers import RobertaForSequenceClassification, RobertaConfig

In [ ]:
import pandas as pd
import numpy as np
import json, re
from tqdm import tqdm_notebook
import random 
import math 


## Torch Modules
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader


In [ ]:
config = RobertaConfig.from_pretrained('roberta-base')
config.num_labels = 2
config

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.15.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification(config)

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [ ]:
SEED = 42

In [ ]:
def reset_randomness(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)  # This implies torch.cuda.manual_seed_all(SEED) now
    # if torch.cuda.is_available(): torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True  # About 15% slower but...
    torch.backends.cudnn.benchmark = False

In [ ]:
def insert_tags(df):
    df = df.assign(
        tagged_paragraph=df[['paragraph', 'offset_start']].apply(
            lambda x: x[0][:x[1]] + ' xxnum ' + x[0][x[1]:], axis=1
        )
    )
    df.tagged_paragraph = df[['tagged_paragraph', 'category']].apply(
        lambda x: re.sub(f'(\$({x[1]}))', r'$ xxtag \2', x[0]), axis=1
    )
    df.tagged_paragraph = '<s> ' + df.tagged_paragraph + ' </s>'
    return df

In [ ]:
def yield_double_redaction(
    txt,
    untouchable_token_set,
    redact_change=0.2,
    unk_chance=0.2,
    mask='<mask>',
    unk='<unk>',
    token_delimiter=' ',
    seed=SEED
):
    random.seed(seed)
    chosen_idx_set_list = []
    tkns = txt.split(token_delimiter)
    tkn_idx_set = {
        tkn_idx for tkn_idx, tkn in enumerate(tkns)
        if tkn not in untouchable_token_set
    }
    n_tkn = len(tkn_idx_set)
    n_sample = math.ceil(n_tkn * redact_change)
    while n_sample <= n_tkn:
        chosen_idx_set = set(random.sample(sorted(tkn_idx_set), k=n_sample))
        chosen_idx_set_list += [chosen_idx_set]
        tkn_idx_set -= chosen_idx_set
        n_tkn = len(tkn_idx_set)
    for idx_set in chosen_idx_set_list:
        dr_tkns = tkns.copy()
        for chosen_idx in idx_set:
            noise = unk if random.random() < unk_chance else mask
            dr_tkns[chosen_idx] = noise
        yield token_delimiter.join(dr_tkns)


def double_redact_tagged_finnum_df(df, special_token_set):
    duplicated_rows = []
    for n, row in df.iterrows():
        for redacted_tagged_paragraph in yield_double_redaction(
            row['tagged_paragraph'],
            {row['target_num'], row['category']} | special_token_set, 
        ):
            row['tagged_paragraph'] = redacted_tagged_paragraph
            duplicated_rows += [row.copy()]
    return pd.DataFrame(duplicated_rows)

In [ ]:
# trn_df = pd.read_json('/content/drive/MyDrive/NTPUIM/NTCIR/FinNUM 3/Dataset/FinNum-3_ConCall_train.json')
trn_df = pd.read_json('/content/drive/MyDrive/FinNUM_dataset/FinNum-3_ConCall_train.json')
trn_df = insert_tags(trn_df)
trn_df = trn_df.assign(is_vld=False)
dr_trn_df = double_redact_tagged_finnum_df(
    trn_df, {'xxtag', 'xxnum', '<s>', '</s>'}
)

In [ ]:
vld_df = pd.read_json('/content/drive/MyDrive/FinNUM_dataset/FinNum-3_ConCall_dev.json')
vld_df = insert_tags(vld_df)
vld_df = vld_df.assign(is_vld=True)

In [ ]:
dr_trn_df

,paragraph,target_num,category,offset_start,offset_end,claim,tagged_paragraph,is_vld
0,Good day and welcome to the Apple Inc. Third Q...,2018.0,date,65,69,0,<s> <mask> day and <mask> to the Apple <mask> ...,False
0,Good day and welcome to the Apple Inc. Third Q...,2018.0,date,65,69,0,<s> Good day <mask> welcome to the <mask> Inc....,False
0,Good day and welcome to the Apple Inc. Third Q...,2018.0,date,65,69,0,<s> Good <mask> and welcome <mask> the Apple I...,False
0,Good day and welcome to the Apple Inc. Third Q...,2018.0,date,65,69,0,<s> Good day and welcome to <mask> Apple Inc. ...,False
1,Please note that some of the information you'l...,10.0,other,504,506,0,<s> Please note that <mask> <mask> the informa...,False
...,...,...,...,...,...,...,...,...
8335,In care delivery our clinical leaders are appl...,99.0,quantity_relative,212,214,0,<s> In care delivery our clinical leaders are ...,False
8336,In care delivery our clinical leaders are appl...,4.0,other,297,298,0,<s> In care delivery <mask> clinical <unk> <ma...,False
8336,In care delivery our clinical leaders are appl...,4.0,other,297,298,0,<s> In <unk> <mask> our clinical leaders are a...,False
8336,In care delivery our clinical leaders are appl...,4.0,other,297,298,0,<s> <mask> care delivery our clinical leaders ...,False


In [ ]:
from collections import Counter, defaultdict


print('claim_num: ', Counter(dr_trn_df['claim']))

claim_num:  Counter({0: 30650, 1: 4333})


In [ ]:
vld_df

,paragraph,target_num,category,offset_start,offset_end,claim,tagged_paragraph,is_vld
0,In care delivery our clinical leaders are appl...,80.0,other,373,375,0,<s> In care delivery our clinical leaders are ...,True
1,We achieved significantly lower total medical ...,30.0,relative,139,141,0,<s> We achieved significantly lower total medi...,True
2,By 2030 there will be over 80 million people i...,2030.0,date,3,7,0,<s> By xxnum 2030 there will be over 80 milli...,True
3,By 2030 there will be over 80 million people i...,80.0,quantity_absolute,27,29,1,<s> By 2030 there will be over xxnum 80 milli...,True
4,By 2030 there will be over 80 million people i...,3.0,quantity_absolute,62,63,0,<s> By 2030 there will be over 80 million peop...,True
...,...,...,...,...,...,...,...,...
1186,Now let's move to a summary of the consolidate...,126.0,money,220,223,0,<s> Now let's move to a summary of the consoli...,True
1187,On a comparable basis excluding divestitures a...,2.0,relative,115,116,0,<s> On a comparable basis excluding divestitur...,True
1188,On a comparable basis excluding divestitures a...,45.1,absolute,226,230,0,<s> On a comparable basis excluding divestitur...,True
1189,On a comparable basis excluding divestitures a...,35.7,absolute,283,287,0,<s> On a comparable basis excluding divestitur...,True


In [ ]:
train_size = 0.8
train_data=df_train.sample(frac=train_size,random_state=200).reset_index(drop=True)
test_data=df_train.drop(train_data.index).reset_index(drop=True)

In [ ]:
def prepare_features(seq_1, max_seq_length = 512, 
             zero_pad = True, include_CLS_token = True, include_SEP_token = True):
    ## Tokenzine Input
    tokens_a = tokenizer.tokenize(seq_1)

    ## Truncate
    if len(tokens_a) > max_seq_length - 2:
        tokens_a = tokens_a[0:(max_seq_length - 2)]
    ## Initialize Tokens
    tokens = []
    if include_CLS_token:
        tokens.append(tokenizer.cls_token)
    ## Add Tokens and separators
    for token in tokens_a:
        tokens.append(token)

    if include_SEP_token:
        tokens.append(tokenizer.sep_token)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    ## Input Mask 
    input_mask = [1] * len(input_ids)
    ## Zero-pad sequence lenght
    if zero_pad:
        while len(input_ids) < max_seq_length:
            input_ids.append(0)
            input_mask.append(0)
    return torch.tensor(input_ids).unsqueeze(0), input_mask

In [ ]:
class Intents(Dataset):
    def __init__(self, dataframe):
        self.len = len(dataframe)
        self.data = dataframe
        
    def __getitem__(self, index):
        tagged_paragraph	 = self.data.tagged_paragraph[index]
        claim = self.data.claim[index]
        X, _  = prepare_features(tagged_paragraph)
        y = self.data.claim[index]
        return X, y
    
    def __len__(self):
        return self.len

In [ ]:
testing_set = Intents(vld_df)

In [ ]:
training_set = Intents(dr_trn_df)
# print(training_set.__getitem__(0)[0].shape)

In [ ]:
training_set.__getitem__(0)

TypeError: ignored

In [ ]:
model(training_set.__getitem__(0)[0])

SequenceClassifierOutput([('logits',
                           tensor([[-0.3340, -0.0336]], grad_fn=<AddmmBackward0>))])

In [ ]:
testing_set = Intents(test_data)
print(testing_set.__getitem__(0)[0].shape)

torch.Size([1, 512])


In [ ]:
training_loader = DataLoader(training_set, batch_size = 2, shuffle = True, drop_last = False)
testing_loader = DataLoader(testing_set, batch_size = 2, shuffle = False, drop_last = False) #不超過16(4-16)

In [ ]:
#BCEWithLogitsLoss 
loss_function = nn.BCEWithLogitsLoss() #CrossEntropyLoss()
learning_rate = 1e-05
optimizer = optim.Adam(params =  model.parameters(), lr=learning_rate)

In [ ]:
inp = training_set.__getitem__(0)[0]
output = model(inp)[0]
print(output.shape)

torch.Size([1, 2])


In [ ]:
model = model.cuda()

In [ ]:
n = 0
for i, (sent, label) in enumerate(training_loader):
  total =0
  if n<1: 
    # a = sent.squeeze(1)
    # print(a)
    # print(a.shape)
    # pre = nn.Sigmoid()(a)
    # print('pre',pre)
    # print('pre_shape:',pre.shape)
    print('label:',label)
    print('label_shape:',label.shape)
    print(label.size())
    total += label.size(0)
    print(total)


  else:
    break

  n+=1

KeyError: ignored

In [ ]:
max_epochs = 3
model = model.train()
loss_function = nn.BCELoss()
m = nn.Sigmoid()

for epoch in tqdm_notebook(range(max_epochs)):
    print("EPOCH -- {}".format(epoch))
    for i, (sent, label) in enumerate(training_loader):
        # print('sent: ',sent)
        # print('Label:', label)
        optimizer.zero_grad()
        sent = sent.squeeze(1)
        if torch.cuda.is_available():
          sent = sent.cuda()
          label = label.cuda()
          output = model.forward(sent)[0]
          _, predicted = torch.max(output, 1)
          # print('output:', output.shape)
          # print('label:', label)
          label = F.one_hot(label, num_classes = 2)
          loss = loss_function(m(output), label.type(torch.float32))
          loss.backward()
          optimizer.step()
        
        if i%100 == 0:
            TP = 0
            TN = 0
            FP = 0
            FN = 0
            total = 0
            for sent, label in testing_loader:
                sent = sent.squeeze(1)
                if torch.cuda.is_available():
                  sent = sent.cuda()
                  label = label.cuda()
                output = model.forward(sent)[0]
                _, predicted = torch.max(output.data, 1)
                total += label.size(0)

                # TP    predict 和 label 同時爲1
                TP += ((predicted == 1) & (label == 1)).cpu().sum()
                # TN    predict 和 label 同時爲0
                TN += ((predicted == 0) & (label == 0)).cpu().sum()
                # FN    predict 0 label 1
                FN += ((predicted == 0) & (label == 1)).cpu().sum()
                # FP    predict 1 label 0
                FP += ((predicted == 1) & (label == 0)).cpu().sum()

                # correct += (predicted.cpu() == label.cpu()).sum()
            # accuracy = 100.00 * correct.numpy() / total
            p = TP / (TP + FP)
            r = TP / (TP + FN)
            F1 = 2 * r * p / (r + p)
            acc = (TP + TN) / (TP + TN + FP + FN)
            print('Iteration: {}. Loss: {}. Accuracy: {}%. TP:{}. TN: {}. FP: {}. FN: {}' .format(i, loss.item(), acc, TP, TN, FP, FN))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/3 [00:00<?, ?it/s]

EPOCH -- 0


KeyError: ignored

In [ ]:
def get_reply(msg):
  model.eval()
  input_msg, _ = prepare_features(msg)
  if torch.cuda.is_available():
    input_msg = input_msg.cuda()
  output = model(input_msg)[0]
  _, pred_label = torch.max(output.data, 1)
  prediction=list(label_to_ix.keys())[pred_label]
  return prediction